Scraping for playing probability for the current Serie A matchday.

Data from http://fantacalcio.it

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv


In [2]:
res = requests.get('https://www.fantacalcio.it/probabili-formazioni-serie-a')

In [3]:
soup = BeautifulSoup(res.text)

groups = soup.findAll('ul', attrs = {'class' : 'player-list'})

probables = pd.DataFrame(columns = ['player', 'starter', 'percentage'])

for i in range (len(groups)):
    players = groups[i].findAll('a', attrs = {'class' : 'player-name'})
    bars = groups[i].findAll('div', attrs = {'class' : 'progress-bar'})
    
    starter_string = groups[i]['class'][1]
    if(starter_string == 'starters'):
        starter = 1
    else:
        starter = 0
    
    for j in range(len(players)):
        player = players[j].contents[1].contents[0]
        perc = float(bars[j]['aria-valuenow'])
        
        row_df = pd.DataFrame([[player, starter, perc]], columns = probables.columns)
        
        probables = pd.concat([probables, row_df], ignore_index = True)
    
probables   

,player,starter,percentage
0,Szczesny,1,80.0
1,Danilo,1,90.0
2,Bremer,1,90.0
3,Gatti,1,60.0
4,Weah,1,80.0
...,...,...,...
474,Fabbian,0,55.0
475,Aebischer,0,60.0
476,El Azzouzi,0,50.0
477,Urbanski,0,15.0


In [4]:
probables = probables.set_index('player')

In [5]:
groups = soup.findAll('ul', attrs = {'class' : 'ballot-list'})

ballots = pd.DataFrame(columns = ['player1', 'player2', 'percentage'])
for i in range (len(groups)):
    players = groups[i].findAll('a', attrs = {'class' : 'player-name'})
    percs = groups[i].findAll('strong', attrs = {'class' : 'percentage'})
    
    player1 = players[0].contents[1].contents[0]
    player2 = players[1].contents[1].contents[0]
    perc = float(percs[0].contents[0].replace('%', ''))
    
    row_df = pd.DataFrame([[player1, player2, perc]], columns = ballots.columns)
        
    ballots = pd.concat([ballots, row_df], ignore_index = True)
                                  
ballots

,player1,player2,percentage
0,Fagioli,Miretti,60.0
1,Kostic,Cambiaso,55.0
2,Gatti,Alex Sandro,60.0
3,Kamada,Guendouzi,55.0
4,De Vrij,Acerbi,60.0
5,Mkhitaryan,Frattesi,55.0
6,Dumfries,Cuadrado,65.0
7,Giroud,Okafor,55.0
8,Krunic,Musah,65.0
9,Badelj,Thorsby,60.0


In [6]:
for i in range(ballots.shape[0]):
    p1 = ballots['player1'][i]
    p2 = ballots['player2'][i]
    
    perc = float(ballots['percentage'][i])
    
    probables.at[p1, 'starter'] = perc/100
    probables.at[p2, 'starter'] = 1 - perc/100
    
probables

,starter,percentage
player,,
Szczesny,1,80.0
Danilo,1,90.0
Bremer,1,90.0
Gatti,0.6,60.0
Weah,1,80.0
...,...,...
Fabbian,0,55.0
Aebischer,0,60.0
El Azzouzi,0,50.0


In [7]:
probables.to_excel('mid_outputs/match_probable_players.xlsx')   